In [1]:
import os
import datetime
import glob
import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import requests
from pymongo import MongoClient

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

mongo_name = ['Australia', 'Bahrain', 'Brazil', 'Brunei', 'Cambodia','Canada', 'Hong Kong',
                'China', 'Czech', 'Denmark', 'Euro', 'Fiji', 'Hungary', 'India', 'Indonesia', 'Israel',
                'Japan', 'Jordan', 'Kuwait', 'Malaysia', 'Mexico', 'New Zealand',
                'Norway', 'Philippines', 'Poland', 'Russia', 'Saudi Arabia', 'Singapore', 'South Africa',
                'Sweden', 'Switzerland', 'Taiwan', 'Thailand', 'Turkey', 'UAE', 'UK', 'USA', 'Vietnam']

In [16]:
class preprocess():
    def __init__(self):
        self.client = MongoClient('localhost', 27017)
        self.db_ex = self.client['exchange']
        self.db_int = self.client['interest']
        self.db_inf = self.client['inflation']
        self.db_m1 = self.client['liquidity']
        
    def monthly_preprocessing(self, country):
        ex_ca = pd.DataFrame(list(self.db_ex[country].find({}, {'_id':0})))
        ex_ca['date'] = pd.to_datetime(ex_ca['date'])
        ex_ca = ex_ca.set_index('date')
        ex_ca['ex'] = np.log(ex_ca['buy'])
        ex_ca = ex_ca.resample(rule = 'M').last()
        
        df = ex_ca.reset_index()[['date','ex']]
        coll_int = self.db_int[country]
        inter = pd.DataFrame(list(coll_int.find({}, {'_id':0})))
        if '1' in inter.columns[1]:
            int_ko = pd.DataFrame(list(self.db_int['Korea1Y'].find({}, {'_id':0})))
            interest = pd.merge(inter, int_ko, on='date', how='inner')
            interest['int_spread'] = interest['interest1Y_y'] - interest['interest1Y_x'] 
            interest['date'] = pd.to_datetime(interest['date'])
            interest = interest[['date', 'int_spread']]

            df = pd.merge(df, interest, on='date', how = 'inner')
            df['date'] = df['date'].dt.strftime("%Y-%m")
        elif '2' in inter.colunns[1]:
            int_ko = pd.DataFrame(list(self.db_int['Korea2Y'].find({}, {'_id':0})))
            interest = pd.merge(inter, int_ko, on='date', how='inner')
            interest['int_spread'] = interest['interest1Y_y'] - interest['interest1Y_x'] 
            interest['date'] = pd.to_datetime(interest['date'])
            interest = interest[['date', 'int_spread']]

            df = pd.merge(df, interest, on='date', how = 'inner')
            df['date'] = df['date'].dt.strftime("%Y-%m")
        elif len(inter) == 0:
            pass

        inf = pd.DataFrame(list(self.db_inf[country + 'M'].find({}, {'_id':0})))
        infq = pd.DataFrame(list(self.db_inf[country + 'Q'].find({}, {'_id':0})))
        inflation_KOR = pd.DataFrame(list(self.db_inf['Korea' + 'M'].find({}, {'_id':0})))
        if len(inf) != 0:
            inf['inf_c'] = inf['inflation'].astype('float').pct_change()*100
            inflation_KOR['inf_kc'] = inflation_KOR['inflation'].astype('float').pct_change()*100
            inflation = pd.merge(inf, inflation_KOR[['date', 'inf_kc']], on='date', how='inner' )
            inflation['inf_spread'] = inflation['inf_kc'] - inflation['inf_c']
            df = pd.merge(df, inflation[['date', 'inf_spread']], on='date', how = 'inner')
        elif len(inf) == 0 and len(infq) != 0:
            infq['date'] = pd.to_datetime(infq['date']).dt.to_period('Q')
            infq = infq.set_index('date').astype('float')
            infq = infq.resample(rule = 'M').fillna(method= 'ffill')
            infq = infq.reset_index()
            infq['date'] = infq['date'].astype('str')
            infq['inf_c'] = infq['inflation'].astype('float').pct_change()*100
            inflation_KOR['inf_kc'] = inflation_KOR['inflation'].astype('float').pct_change()*100
            inflation = pd.merge(infq, inflation_KOR[['date', 'inf_kc']], on='date', how='inner' )
            inflation['inf_spread'] = inflation['inf_kc'] - inflation['inf_c']
            df = pd.merge(df, inflation[['date', 'inf_spread']], on='date', how = 'inner')
        elif len(inf) == 0 and len(infq) == 0:
            pass

        
        m1 = pd.DataFrame(list(self.db_m1[country + 'M'].find({}, {'_id':0})))
        m1q = pd.DataFrame(list(self.db_m1[country + 'Q'].find({}, {'_id':0})))
        if len(m1) != 0:
            m1_KOR = pd.DataFrame(list(self.db_m1['Korea' + 'M'].find({}, {'_id':0})))
            m1['m1'] = np.log(m1['liquidity'].astype('float'))
            m1_KOR['m1_k'] = np.log(m1_KOR['liquidity'].astype('float'))
            m1 = pd.merge(m1[['date','m1']], m1_KOR[['date', 'm1_k']], on='date', how='inner')
            m1['m1_spread'] = m1['m1_k'] - m1['m1']
            df = pd.merge(df, m1[['date', 'm1_spread']], on='date', how = 'inner')
        elif len(m1) == 0 and len(m1q) != 0:
            m1_KOR = pd.DataFrame(list(self.db_m1['Korea' + 'M'].find({}, {'_id':0})))
            m1q['date'] = pd.to_datetime(m1q['date']).dt.to_period('Q')
            m1q = m1q.set_index('date').astype('float')
            m1q = m1q.resample(rule = 'M').fillna(method= 'ffill')
            m1q = m1q.reset_index()
            m1q['date'] = m1q['date'].astype('str')
            m1q['m1'] = np.log(m1q['liquidity'].astype('float'))
            m1_KOR['m1_k'] = np.log(m1_KOR['liquidity'].astype('float'))
            m1q = pd.merge(m1q[['date','m1']], m1_KOR[['date', 'm1_k']], on='date', how='inner')
            m1q['m1_spread'] = m1q['m1_k'] - m1q['m1']
            df = pd.merge(df, m1q[['date', 'm1_spread']], on='date', how = 'inner')
        elif len(m1) == 0 and len(m1q) == 0:
            pass
        
        date_time = pd.to_datetime(df.pop('date'), format='%Y-%m')
        df = df.astype('float')
        
        return df, date_time
    
    def daily_preprocessing(self, country):
        ex_ca = pd.DataFrame(list(self.db_ex[country].find({}, {'_id':0})))
        ex_ca['date'] = pd.to_datetime(ex_ca['date'])
        ex_ca['ex'] = np.log(ex_ca['buy'])
        df = ex_ca[['date','ex']]
        
        coll_int = self.db_int[country]
        inter = pd.DataFrame(list(coll_int.find({}, {'_id':0})))
        if '1' in inter.columns[1]:
            int_ko = pd.DataFrame(list(self.db_int['Korea1Y'].find({}, {'_id':0})))
        elif '2' in inter.colunns[1]:
            int_ko = pd.DataFrame(list(self.db_int['Korea2Y'].find({}, {'_id':0})))

        interest = pd.merge(inter, int_ko, on='date', how='inner')
        interest['int_spread'] = interest['interest1Y_y'] - interest['interest1Y_x'] 
        interest['date'] = pd.to_datetime(interest['date'])
        interest = interest[['date', 'int_spread']]

        df = pd.merge(df, interest, on='date', how = 'inner')
        
        inf = pd.DataFrame(list(self.db_inf[country + 'M'].find({}, {'_id':0})))
        infq = pd.DataFrame(list(self.db_inf[country + 'Q'].find({}, {'_id':0})))
        inflation_KOR = pd.DataFrame(list(self.db_inf['Korea' + 'M'].find({}, {'_id':0})))
        
        if len(inf) != 0:
            inf['inf_c'] = inf['inflation'].astype('float').pct_change()*100
            inflation_KOR['inf_kc'] = inflation_KOR['inflation'].astype('float').pct_change()*100
            print(inflation_KOR)
            inflation = pd.merge(inf, inflation_KOR[['date', 'inf_kc']], on='date', how='inner' )
            print(inflation)
            inflation['inf_spread'] = inflation['inf_kc'] - inflation['inf_c']
            inflation['date'] = pd.to_datetime(inflation['date']).dt.to_period('M')
            inflation = inflation.set_index('date')
            print(inflation)
            inflation = inflation.resample(rule = 'D').fillna(method= 'ffill')
            inflation = inflation.reset_index()
            inflation['date'] = pd.to_datetime(inflation['date'].astype('str'))
            df = pd.merge(df, inflation[['date', 'inf_spread']], on='date', how = 'inner')
        elif len(inf) == 0 and len(infq) != 0:
            infq['date'] = pd.to_datetime(infq['date']).dt.to_period('Q')
            infq = infq.set_index('date').astype('float')
            infq = infq.resample(rule = 'M').fillna(method= 'ffill')
            infq = infq.reset_index()
            infq['date'] = infq['date'].astype('str')
            infq['inf_c'] = infq['inflation'].astype('float').pct_change()*100
            inflation_KOR['inf_kc'] = inflation_KOR['inflation'].astype('float').pct_change()*100
            inflation = pd.merge(infq, inflation_KOR[['date', 'inf_kc']], on='date', how='inner' )
            inflation['inf_spread'] = inflation['inf_kc'] - inflation['inf_c']
            inflation['date'] = pd.to_datetime(inflation['date']).dt.to_period('M')
            inflation = inflation.set_index('date')
            inflation = inflation.resample(rule = 'D').fillna(method= 'ffill')
            inflation = inflation.reset_index()
            inflation['date'] = pd.to_datetime(inflation['date'].astype('str'))
            df = pd.merge(df, inflation[['date', 'inf_spread']], on='date', how = 'inner')
        elif len(inf) == 0 and len(infq) == 0:
            pass
        
        m1 = pd.DataFrame(list(self.db_m1[country + 'M'].find({}, {'_id':0})))
        m1q = pd.DataFrame(list(self.db_m1[country + 'Q'].find({}, {'_id':0})))
        if len(m1) != 0:
            m1_KOR = pd.DataFrame(list(self.db_m1['Korea' + 'M'].find({}, {'_id':0})))
            print(m1_KOR)
            m1['m1'] = np.log(m1['liquidity'].astype('float'))
            m1_KOR['m1_k'] = np.log(m1_KOR['liquidity'].astype('float'))
            m1 = pd.merge(m1[['date','m1']], m1_KOR[['date', 'm1_k']], on='date', how='inner')
            m1['m1_spread'] = m1['m1_k'] - m1['m1']
            m1['date'] = pd.to_datetime(m1['date']).dt.to_period('M')
            m1 = m1.set_index('date')
            print(m1)
            m1 = m1.resample(rule = 'D').fillna(method= 'ffill')
            m1 = m1.reset_index()
            m1['date'] = pd.to_datetime(m1['date'].astype('str'))
            df = pd.merge(df, m1[['date', 'm1_spread']], on='date', how = 'inner')
        elif len(m1) == 0 and len(m1q) != 0:
            m1_KOR = pd.DataFrame(list(self.db_m1['Korea' + 'M'].find({}, {'_id':0})))
            m1q['date'] = pd.to_datetime(m1q['date']).dt.to_period('Q')
            m1q = m1q.set_index('date').astype('float')
            m1q = m1q.resample(rule = 'M').fillna(method= 'ffill')
            m1q = m1q.reset_index()
            m1q['date'] = m1q['date'].astype('str')
            m1q['m1'] = np.log(m1q['liquidity'].astype('float'))
            m1_KOR['m1_k'] = np.log(m1_KOR['liquidity'].astype('float'))
            m1q = pd.merge(m1q[['date','m1']], m1_KOR[['date', 'm1_k']], on='date', how='inner')
            m1q['m1_spread'] = m1q['m1_k'] - m1q['m1']
            m1q['date'] = pd.to_datetime(m1q['date']).dt.to_period('M')
            m1q = m1q.set_index('date')
            m1q = m1q.resample(rule = 'D').fillna(method= 'ffill')
            m1q = m1q.reset_index()
            m1q['date'] = pd.to_datetime(m1q['date'].astype('str'))
            df = pd.merge(df, m1q[['date', 'm1_spread']], on='date', how = 'inner')
        elif len(m1) == 0 and len(m1q) == 0:
            pass
        
        date_time = pd.to_datetime(df.pop('date'), format='%Y-%m-%d')
        df = df.astype('float')
        
        return df, date_time
    
class lstm():
    def __init__(self):
        pass
    
    def model_run(self, df, unit):
        mean = df.mean()
        std = df.std()

        df = (df - mean) / std
        
        dfx = df# 반복횟수 600
        dfy = df['ex']

        window_size = 5
        data_size = 4

        x = dfx.values.tolist()
        y = dfy.values.tolist()

        data_x = []
        data_y = []
        for i in range(len(y) - window_size):
            _x = x[i : i + window_size] # 다음 날 종가(i+windows_size)는 포함되지 않음
            _y = y[i + window_size]     # 다음 날 종가
            data_x.append(_x)
            data_y.append(_y)
        print(_x, "->", _y)

        train_size = int(len(data_y) * 0.9)
        train_x = np.array(data_x[0 : train_size])
        train_y = np.array(data_y[0 : train_size])

        test_size = len(data_y) - train_size
        test_x = np.array(data_x[train_size : len(data_x)])
        test_y = np.array(data_y[train_size : len(data_y)])

        # 모델 생성
        model = Sequential()
        model.add(LSTM(units=5, activation='tanh', return_sequences=True, input_shape=(window_size, data_size)))
        model.add(LSTM(units=5, activation='tanh'))
        model.add(Dense(units=1))
        model.summary()

        optimizer = optimizers.Adam(lr = 0.001)
        model.compile(optimizer=optimizer, loss='mean_squared_error')
        # model.compile(optimizer=adam, loss='mean_squared_error')
        history = model.fit(train_x, train_y, epochs=300, batch_size=10, validation_split=0.1)
        pred_y = model.predict(test_x)

        return np.exp(pred_y[-1])
    
if __name__=='__main__':
    pre = preprocess()
    df = pre.daily_preprocessing('Canada')
    print(df)

        date          inflation     inf_kc
0    1951-08  0.147918670258338        NaN
1    1951-09  0.146434404402627  -1.003434
2    1951-10  0.127205172932795 -13.131635
3    1951-11  0.128689438788506   1.166828
4    1951-12  0.140519454457043   9.192686
..       ...                ...        ...
856  2022-11         130.935933   1.027847
857  2022-12         133.043332   1.609488
858  2023-01         135.752426   2.036249
859  2023-02         138.929323   2.340214
860  2023-03         143.870904   3.556903

[861 rows x 3 columns]
        date         inflation     inf_c     inf_kc
0    1951-08  12.0206067544362  0.719424        NaN
1    1951-09  12.1064682312536  0.714286  -1.003434
2    1951-10   12.192329708071  0.709220 -13.131635
3    1951-11  12.2781911848884  0.704225   1.166828
4    1951-12  12.2781911848884  0.000000   9.192686
..       ...               ...       ...        ...
856  2022-11         138.06195  1.139246   1.027847
857  2022-12        140.194754  1.544817   1

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [40]:
client = MongoClient('localhost', 27017)
db_ex = client['exchange']
db_int = client['interest']
db_inf = client['inflation']
db_m1 = client['liquidity']

m1 = pd.DataFrame(list(db_m1['Canada' + 'M'].find({}, {'_id':0})))
m1q = pd.DataFrame(list(db_m1['Canada' + 'Q'].find({}, {'_id':0})))
m1_KOR = pd.DataFrame(list(db_m1['Korea' + 'M'].find({}, {'_id':0})))
m1['m1'] = np.log(m1['liquidity'].astype('float'))
m1_KOR['m1_k'] = np.log(m1_KOR['liquidity'].astype('float'))
liq = pd.merge(m1[['date','m1']], m1_KOR[['date', 'm1_k']], on='date', how='inner')
liq
# if len(m1) != 0:
#     m1_KOR = pd.DataFrame(list(db_m1['Korea' + 'M'].find({}, {'_id':0})))
#     print(m1_KOR)
#     m1['m1'] = np.log(m1['liquidity'].astype('float'))
#     m1_KOR['m1_k'] = np.log(m1_KOR['liquidity'].astype('float'))
#     m1 = pd.merge(m1[['date','m1']], m1_KOR[['date', 'm1_k']], on='date', how='inner')
#     m1['m1_spread'] = m1['m1_k'] - m1['m1']
#     m1['date'] = pd.to_datetime(m1['date']).dt.to_period('M')
#     m1 = m1[['date','m1_spread']]
#     m1 = m1.set_index('date')
#     print(m1)
#     m1 = m1.resample(rule = 'D').last().fillna(method= 'ffill')
# m1
#     m1 = m1.reset_index()
#     m1['date'] = pd.to_datetime(m1['date'].astype('str'))
#     df = pd.merge(df, m1[['date', 'm1_spread']], on='date', how = 'inner')
#         elif len(m1) == 0 and len(m1q) != 0:
#             m1_KOR = pd.DataFrame(list(self.db_m1['Korea' + 'M'].find({}, {'_id':0})))
#             m1q['date'] = pd.to_datetime(m1q['date']).dt.to_period('Q')
#             m1q = m1q.set_index('date').astype('float')
#             m1q = m1q.resample(rule = 'M').fillna(method= 'ffill')
#             m1q = m1q.reset_index()
#             m1q['date'] = m1q['date'].astype('str')
#             m1q['m1'] = np.log(m1q['liquidity'].astype('float'))
#             m1_KOR['m1_k'] = np.log(m1_KOR['liquidity'].astype('float'))
#             m1q = pd.merge(m1q[['date','m1']], m1_KOR[['date', 'm1_k']], on='date', how='inner')
#             m1q['m1_spread'] = m1q['m1_k'] - m1q['m1']
#             m1q['date'] = pd.to_datetime(m1q['date']).dt.to_period('M')
#             m1q = m1q.set_index('date')
#             m1q = m1q.resample(rule = 'D').fillna(method= 'ffill')
#             m1q = m1q.reset_index()
#             m1q['date'] = pd.to_datetime(m1q['date'].astype('str'))
#             df = pd.merge(df, m1q[['date', 'm1_spread']], on='date', how = 'inner')
#         elif len(m1) == 0 and len(m1q) == 0:
#             pass

,date,m1,m1_k
0,1950-12,7.142835,3.230804
1,1951-12,6.969800,3.605498
2,1952-12,6.964995,4.399375
3,1953-12,6.827207,4.675629
4,1954-12,6.871413,4.658711
...,...,...,...
800,2022-11,11.771701,13.062142
801,2022-12,11.936640,13.100425
802,2023-01,11.856852,13.116184
803,2023-02,11.911429,13.137707


In [158]:
ex_ca = ex_ca.resample(rule = 'M').last()
df = ex_ca.reset_index()[['date','ex']]
df

,date,ex
0,2003-09-30,6.767631
1,2003-10-31,6.821772
2,2003-11-30,6.843889
3,2003-12-31,6.845922
4,2004-01-31,6.801283
...,...,...
221,2022-02-28,6.867090
222,2022-03-31,6.894559
223,2022-04-30,6.915029
224,2022-05-31,6.904349


In [159]:
coll_int = db_int[mongo_name[5]]
inter = pd.DataFrame(list(coll_int.find({}, {'_id':0})))
if '1' in inter.columns[1]:
    int_ko = pd.DataFrame(list(db_int['Korea1Y'].find({}, {'_id':0})))
elif '2' in inter.colunns[1]:
    int_ko = pd.DataFrame(list(db_int['Korea2Y'].find({}, {'_id':0})))

interest = pd.merge(inter, int_ko, on='date', how='inner')
interest['int_spread'] = interest['interest1Y_y'] - interest['interest1Y_x'] 
interest['date'] = pd.to_datetime(interest['date'])
interest = interest[['date', 'int_spread']]

df = pd.merge(df, interest, on='date', how = 'inner')
df['date'] = df['date'].dt.strftime("%Y-%m")
df

,date,ex,int_spread
0,2003-09,6.767631,1.550
1,2003-10,6.821772,1.300
2,2003-11,6.843889,1.740
3,2003-12,6.845922,1.980
4,2004-01,6.801283,2.160
...,...,...,...
220,2022-01,6.874426,0.468
221,2022-02,6.867090,0.256
222,2022-03,6.894559,-0.285
223,2022-04,6.915029,-0.269


In [169]:
inf = pd.DataFrame(list(db_inf[mongo_name[0] + 'M'].find({}, {'_id':0})))
infq = pd.DataFrame(list(db_inf[mongo_name[0] + 'Q'].find({}, {'_id':0})))
inflation_KOR = pd.DataFrame(list(db_inf['Korea' + 'M'].find({}, {'_id':0})))
if len(inf) != 0:
    inf['inf_c'] = inf['inflation'].astype('float').pct_change()*100
    inflation_KOR['inf_kc'] = inflation_KOR['inflation'].astype('float').pct_change()*100
    inflation = pd.merge(inf, inflation_KOR[['date', 'inf_kc']], on='date', how='inner' )
    inflation['inf_spread'] = inflation['inf_kc'] - inflation['inf_c']
    df = pd.merge(df, inflation[['date', 'inf_spread']], on='date', how = 'inner')
elif len(inf) == 0 and len(infq) != 0:
    infq['date'] = pd.to_datetime(infq['date']).dt.to_period('Q')
    infq = infq.set_index('date').astype('float')
    infq = infq.resample(rule = 'M').fillna(method= 'ffill')
    infq = infq.reset_index()
    infq['date'] = infq['date'].astype('str')
    infq['inf_c'] = infq['inflation'].astype('float').pct_change()*100
    inflation_KOR['inf_kc'] = inflation_KOR['inflation'].astype('float').pct_change()*100
    inflation = pd.merge(infq, inflation_KOR[['date', 'inf_kc']], on='date', how='inner' )
    inflation['inf_spread'] = inflation['inf_kc'] - inflation['inf_c']
    df = pd.merge(df, inflation[['date', 'inf_spread']], on='date', how = 'inner')
elif len(inf) == 0 and len(infq) == 0:
    pass

/var/folders/v0/twgc0q1d1qnc66ygdv3d817r0000gn/T/ipykernel_1929/500346845.py:20: FutureWarning: Passing 'suffixes' which cause duplicate columns {'inf_spread_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = pd.merge(df, inflation[['date', 'inf_spread']], on='date', how = 'inner')


In [168]:
m1 = pd.DataFrame(list(db_m1[mongo_name[-3] + 'M'].find({}, {'_id':0})))
if len(m1) == 0:
    m1 = pd.DataFrame(list(db_m1[mongo_name[-3] + 'Q'].find({}, {'_id':0})))

m1_KOR = pd.DataFrame(list(db_m1['Korea' + 'M'].find({}, {'_id':0})))
m1['date'] = pd.to_datetime(m1['date']).dt.to_period('Q')
m1 = m1.set_index('date').astype('float')
m1 = m1.resample(rule = 'M').fillna(method= 'ffill')
m1
# m1 = m1.reset_index()
# m1['date'] = m1['date'].astype('str')
# m1['m1'] = np.log(m1['liquidity'].astype('float'))
# m1_KOR['m1_k'] = np.log(m1_KOR['liquidity'].astype('float'))
# m1 = pd.merge(m1[['date','m1']], m1_KOR[['date', 'm1_k']], on='date', how='inner')
# m1['m1_spread'] = m1['m1_k'] - m1['m1']
# df = pd.merge(df, m1[['date', 'm1_spread']], on='date', how = 'inner')
# df

,liquidity
date,
1950-10,581.260003
1950-11,581.260003
1950-12,581.260003
1951-01,581.260003
1951-02,581.260003
...,...
2021-11,176024.004962
2021-12,176024.004962
2022-01,171196.649348


In [217]:
ex_ca = pd.DataFrame(list(db_ex[mongo_name[5]].find({}, {'_id':0})))
ex_ca['date'] = pd.to_datetime(ex_ca['date'])
ex_ca['ex'] = np.log(ex_ca['buy'])
df = ex_ca[['date','ex']]
df

,date,ex
0,2003-09-24,6.763134
1,2003-09-25,6.768666
2,2003-09-26,6.765420
3,2003-09-29,6.759522
4,2003-09-30,6.767631
...,...,...
4639,2022-05-27,6.911448
4640,2022-05-30,6.902793
4641,2022-05-31,6.904349
4642,2022-06-02,6.915485


In [218]:
coll_int = db_int[mongo_name[5]]
inter = pd.DataFrame(list(coll_int.find({}, {'_id':0})))
if '1' in inter.columns[1]:
    int_ko = pd.DataFrame(list(db_int['Korea1Y'].find({}, {'_id':0})))
elif '2' in inter.colunns[1]:
    int_ko = pd.DataFrame(list(db_int['Korea2Y'].find({}, {'_id':0})))

interest = pd.merge(inter, int_ko, on='date', how='inner')
interest['int_spread'] = interest['interest1Y_y'] - interest['interest1Y_x'] 
interest['date'] = pd.to_datetime(interest['date'])
interest = interest[['date', 'int_spread']]

df = pd.merge(df, interest, on='date', how = 'inner')
df

,date,ex,int_spread
0,2003-09-24,6.763134,1.520
1,2003-09-25,6.768666,1.510
2,2003-09-26,6.765420,1.520
3,2003-09-29,6.759522,1.490
4,2003-09-30,6.767631,1.550
...,...,...,...
4639,2022-05-27,6.911448,-0.188
4640,2022-05-30,6.902793,-0.208
4641,2022-05-31,6.904349,-0.278
4642,2022-06-02,6.915485,-0.448


In [230]:
inf = pd.DataFrame(list(db_inf[mongo_name[0] + 'M'].find({}, {'_id':0})))
infq = pd.DataFrame(list(db_inf[mongo_name[0] + 'Q'].find({}, {'_id':0})))
inflation_KOR = pd.DataFrame(list(db_inf['Korea' + 'M'].find({}, {'_id':0})))
if len(inf) != 0:
    inf['inf_c'] = inf['inflation'].astype('float').pct_change()*100
    inflation_KOR['inf_kc'] = inflation_KOR['inflation'].astype('float').pct_change()*100
    inflation = pd.merge(inf, inflation_KOR[['date', 'inf_kc']], on='date', how='inner' )
    inflation['inf_spread'] = inflation['inf_kc'] - inflation['inf_c']
    inflation['date'] = pd.to_datetime(inflation['date']).dt.to_period('M')
    inflation = inflation.set_index('date')
    inflation = inflation.resample(rule = 'D').fillna(method= 'ffill')
    inflation = inflation.reset_index()
    inflation['date'] = inflation['date'].astype('str')
    df = pd.merge(df, inflation[['date', 'inf_spread']], on='date', how = 'inner')
elif len(inf) == 0 and len(infq) != 0:
    infq['date'] = pd.to_datetime(infq['date']).dt.to_period('Q')
    infq = infq.set_index('date')
    infq = infq.resample(rule = 'M').fillna(method= 'ffill')
    infq = infq.reset_index()
    infq['date'] = infq['date'].astype('str')
    infq['inf_c'] = infq['inflation'].astype('float').pct_change()*100
    inflation_KOR['inf_kc'] = inflation_KOR['inflation'].astype('float').pct_change()*100
    inflation = pd.merge(infq, inflation_KOR[['date', 'inf_kc']], on='date', how='inner' )
    inflation['inf_spread'] = inflation['inf_kc'] - inflation['inf_c']
    inflation['date'] = pd.to_datetime(inflation['date']).dt.to_period('M')
    inflation = inflation.set_index('date')
    inflation = inflation.resample(rule = 'D').fillna(method= 'ffill')
    inflation = inflation.reset_index()
    inflation['date'] = pd.to_datetime(inflation['date'].astype('str'))
    df = pd.merge(df, inflation[['date', 'inf_spread']], on='date', how = 'inner')
df

,date,ex,int_spread,inf_spread_x,inf_spread_y
0,2003-09-24,6.763134,1.520,0.904861,0.904861
1,2003-09-25,6.768666,1.510,0.904861,0.904861
2,2003-09-26,6.765420,1.520,0.904861,0.904861
3,2003-09-29,6.759522,1.490,0.904861,0.904861
4,2003-09-30,6.767631,1.550,0.904861,0.904861
...,...,...,...,...,...
4595,2022-03-25,6.898079,-0.266,0.721747,0.721747
4596,2022-03-28,6.908585,-0.190,0.721747,0.721747
4597,2022-03-29,6.901335,-0.206,0.721747,0.721747
4598,2022-03-30,6.895845,-0.328,0.721747,0.721747


In [241]:
class preprocess():
    def __init__(self):
        self.client = MongoClient('localhost', 27017)
        self.db_ex = self.client['exchange']
        self.db_int = self.client['interst2']
        self.db_inf = self.client['inflation']
        self.db_m1 = self.client['liquidity']
        
    def monthly_preprocessing(self, country):
        ex_ca = pd.DataFrame(list(self.db_ex[country].find({}, {'_id':0})))
        ex_ca['date'] = pd.to_datetime(ex_ca['date'])
        ex_ca = ex_ca.set_index('date')
        ex_ca['ex'] = np.log(ex_ca['buy'])
        ex_ca = ex_ca.resample(rule = 'M').last()
        
        df = ex_ca.reset_index()[['date','ex']]
        coll_int = self.db_int[country]
        inter = pd.DataFrame(list(coll_int.find({}, {'_id':0})))
        if '1' in inter.columns[1]:
            int_ko = pd.DataFrame(list(self.db_int['Korea1Y'].find({}, {'_id':0})))
            interest = pd.merge(inter, int_ko, on='date', how='inner')
            interest['int_spread'] = interest['interest1Y_y'] - interest['interest1Y_x'] 
            interest['date'] = pd.to_datetime(interest['date'])
            interest = interest[['date', 'int_spread']]

            df = pd.merge(df, interest, on='date', how = 'inner')
            df['date'] = df['date'].dt.strftime("%Y-%m")
        elif '2' in inter.colunns[1]:
            int_ko = pd.DataFrame(list(self.db_int['Korea2Y'].find({}, {'_id':0})))
            interest = pd.merge(inter, int_ko, on='date', how='inner')
            interest['int_spread'] = interest['interest1Y_y'] - interest['interest1Y_x'] 
            interest['date'] = pd.to_datetime(interest['date'])
            interest = interest[['date', 'int_spread']]

            df = pd.merge(df, interest, on='date', how = 'inner')
            df['date'] = df['date'].dt.strftime("%Y-%m")
        elif len(inter) == 0:
            pass

        inf = pd.DataFrame(list(self.db_inf[country + 'M'].find({}, {'_id':0})))
        infq = pd.DataFrame(list(self.db_inf[country + 'Q'].find({}, {'_id':0})))
        inflation_KOR = pd.DataFrame(list(self.db_inf['Korea' + 'M'].find({}, {'_id':0})))
        if len(inf) != 0:
            inf['inf_c'] = inf['inflation'].astype('float').pct_change()*100
            inflation_KOR['inf_kc'] = inflation_KOR['inflation'].astype('float').pct_change()*100
            inflation = pd.merge(inf, inflation_KOR[['date', 'inf_kc']], on='date', how='inner' )
            inflation['inf_spread'] = inflation['inf_kc'] - inflation['inf_c']
            df = pd.merge(df, inflation[['date', 'inf_spread']], on='date', how = 'inner')
        elif len(inf) == 0 and len(infq) != 0:
            infq['date'] = pd.to_datetime(infq['date']).dt.to_period('Q')
            infq = infq.set_index('date').astype('float')
            infq = infq.resample(rule = 'M').fillna(method= 'ffill')
            infq = infq.reset_index()
            infq['date'] = infq['date'].astype('str')
            infq['inf_c'] = infq['inflation'].astype('float').pct_change()*100
            inflation_KOR['inf_kc'] = inflation_KOR['inflation'].astype('float').pct_change()*100
            inflation = pd.merge(infq, inflation_KOR[['date', 'inf_kc']], on='date', how='inner' )
            inflation['inf_spread'] = inflation['inf_kc'] - inflation['inf_c']
            df = pd.merge(df, inflation[['date', 'inf_spread']], on='date', how = 'inner')
        elif len(inf) == 0 and len(infq) == 0:
            pass

        
        m1 = pd.DataFrame(list(self.db_m1[country + 'M'].find({}, {'_id':0})))
        m1q = pd.DataFrame(list(self.db_m1[country + 'Q'].find({}, {'_id':0})))
        if len(m1) != 0:
            m1_KOR = pd.DataFrame(list(self.db_m1['Korea' + 'M'].find({}, {'_id':0})))
            m1['m1'] = np.log(m1['liquidity'].astype('float'))
            m1_KOR['m1_k'] = np.log(m1_KOR['liquidity'].astype('float'))
            m1 = pd.merge(m1[['date','m1']], m1_KOR[['date', 'm1_k']], on='date', how='inner')
            m1['m1_spread'] = m1['m1_k'] - m1['m1']
            df = pd.merge(df, m1[['date', 'm1_spread']], on='date', how = 'inner')
        elif len(m1) == 0 and len(m1q) != 0:
            m1_KOR = pd.DataFrame(list(self.db_m1['Korea' + 'M'].find({}, {'_id':0})))
            m1q['date'] = pd.to_datetime(m1q['date']).dt.to_period('Q')
            m1q = m1q.set_index('date').astype('float')
            m1q = m1q.resample(rule = 'M').fillna(method= 'ffill')
            m1q = m1q.reset_index()
            m1q['date'] = m1q['date'].astype('str')
            m1q['m1'] = np.log(m1q['liquidity'].astype('float'))
            m1_KOR['m1_k'] = np.log(m1_KOR['liquidity'].astype('float'))
            m1q = pd.merge(m1q[['date','m1']], m1_KOR[['date', 'm1_k']], on='date', how='inner')
            m1q['m1_spread'] = m1q['m1_k'] - m1q['m1']
            df = pd.merge(df, m1q[['date', 'm1_spread']], on='date', how = 'inner')
        elif len(m1) == 0 and len(m1q) == 0:
            pass
        
        return df
    
    def daily_preprocessing(self, country):
        ex_ca = pd.DataFrame(list(self.db_ex[country].find({}, {'_id':0})))
        ex_ca['date'] = pd.to_datetime(ex_ca['date'])
        ex_ca['ex'] = np.log(ex_ca['buy'])
        df = ex_ca[['date','ex']]
        
        coll_int = self.db_int[country]
        inter = pd.DataFrame(list(coll_int.find({}, {'_id':0})))
        if '1' in inter.columns[1]:
            int_ko = pd.DataFrame(list(self.db_int['Korea1Y'].find({}, {'_id':0})))
        elif '2' in inter.colunns[1]:
            int_ko = pd.DataFrame(list(self.db_int['Korea2Y'].find({}, {'_id':0})))

        interest = pd.merge(inter, int_ko, on='date', how='inner')
        interest['int_spread'] = interest['interest1Y_y'] - interest['interest1Y_x'] 
        interest['date'] = pd.to_datetime(interest['date'])
        interest = interest[['date', 'int_spread']]

        df = pd.merge(df, interest, on='date', how = 'inner')
        
        inf = pd.DataFrame(list(self.db_inf[country + 'M'].find({}, {'_id':0})))
        infq = pd.DataFrame(list(self.db_inf[country + 'Q'].find({}, {'_id':0})))
        inflation_KOR = pd.DataFrame(list(self.db_inf['Korea' + 'M'].find({}, {'_id':0})))
        
        if len(inf) != 0:
            inf['inf_c'] = inf['inflation'].astype('float').pct_change()*100
            inflation_KOR['inf_kc'] = inflation_KOR['inflation'].astype('float').pct_change()*100
            inflation = pd.merge(inf, inflation_KOR[['date', 'inf_kc']], on='date', how='inner' )
            inflation['inf_spread'] = inflation['inf_kc'] - inflation['inf_c']
            inflation['date'] = pd.to_datetime(inflation['date']).dt.to_period('M')
            inflation = inflation.set_index('date')
            inflation = inflation.resample(rule = 'D').fillna(method= 'ffill')
            inflation = inflation.reset_index()
            inflation['date'] = pd.to_datetime(inflation['date'].astype('str'))
            df = pd.merge(df, inflation[['date', 'inf_spread']], on='date', how = 'inner')
        elif len(inf) == 0 and len(infq) != 0:
            infq['date'] = pd.to_datetime(infq['date']).dt.to_period('Q')
            infq = infq.set_index('date').astype('float')
            infq = infq.resample(rule = 'M').fillna(method= 'ffill')
            infq = infq.reset_index()
            infq['date'] = infq['date'].astype('str')
            infq['inf_c'] = infq['inflation'].astype('float').pct_change()*100
            inflation_KOR['inf_kc'] = inflation_KOR['inflation'].astype('float').pct_change()*100
            inflation = pd.merge(infq, inflation_KOR[['date', 'inf_kc']], on='date', how='inner' )
            inflation['inf_spread'] = inflation['inf_kc'] - inflation['inf_c']
            inflation['date'] = pd.to_datetime(inflation['date']).dt.to_period('M')
            inflation = inflation.set_index('date')
            inflation = inflation.resample(rule = 'D').fillna(method= 'ffill')
            inflation = inflation.reset_index()
            inflation['date'] = pd.to_datetime(inflation['date'].astype('str'))
            df = pd.merge(df, inflation[['date', 'inf_spread']], on='date', how = 'inner')
        elif len(inf) == 0 and len(infq) == 0:
            pass
        
        m1 = pd.DataFrame(list(self.db_m1[country + 'M'].find({}, {'_id':0})))
        m1q = pd.DataFrame(list(self.db_m1[country + 'Q'].find({}, {'_id':0})))
        if len(m1) != 0:
            m1_KOR = pd.DataFrame(list(self.db_m1['Korea' + 'M'].find({}, {'_id':0})))
            m1['m1'] = np.log(m1['liquidity'].astype('float'))
            m1_KOR['m1_k'] = np.log(m1_KOR['liquidity'].astype('float'))
            m1 = pd.merge(m1[['date','m1']], m1_KOR[['date', 'm1_k']], on='date', how='inner')
            m1['m1_spread'] = m1['m1_k'] - m1['m1']
            m1['date'] = pd.to_datetime(m1['date']).dt.to_period('M')
            m1 = m1.set_index('date')
            m1 = m1.resample(rule = 'D').fillna(method= 'ffill')
            m1 = m1.reset_index()
            m1['date'] = pd.to_datetime(m1['date'].astype('str'))
            df = pd.merge(df, m1[['date', 'm1_spread']], on='date', how = 'inner')
        elif len(m1) == 0 and len(m1q) != 0:
            m1_KOR = pd.DataFrame(list(self.db_m1['Korea' + 'M'].find({}, {'_id':0})))
            m1q['date'] = pd.to_datetime(m1q['date']).dt.to_period('Q')
            m1q = m1q.set_index('date').astype('float')
            m1q = m1q.resample(rule = 'M').fillna(method= 'ffill')
            m1q = m1q.reset_index()
            m1q['date'] = m1q['date'].astype('str')
            m1q['m1'] = np.log(m1q['liquidity'].astype('float'))
            m1_KOR['m1_k'] = np.log(m1_KOR['liquidity'].astype('float'))
            m1q = pd.merge(m1q[['date','m1']], m1_KOR[['date', 'm1_k']], on='date', how='inner')
            m1q['m1_spread'] = m1q['m1_k'] - m1q['m1']
            m1q['date'] = pd.to_datetime(m1q['date']).dt.to_period('M')
            m1q = m1q.set_index('date')
            m1q = m1q.resample(rule = 'D').fillna(method= 'ffill')
            m1q = m1q.reset_index()
            m1q['date'] = pd.to_datetime(m1q['date'].astype('str'))
            df = pd.merge(df, m1q[['date', 'm1_spread']], on='date', how = 'inner')
        elif len(m1) == 0 and len(m1q) == 0:
            pass
        
        return df

In [239]:
a = preprocess()
a.monthly_preprocessing('UK')

,date,ex,int_spread,inf_spread,m1_spread
0,2003-09,7.578866,0.5087,0.644444,1.029715
1,2003-10,7.624482,0.0432,-0.041898,1.010477
2,2003-11,7.651253,0.4863,-0.178625,1.058370
3,2003-12,7.680365,0.7916,0.190798,1.091175
4,2004-01,7.683510,0.3080,1.013010,1.090901
...,...,...,...,...,...
218,2021-11,7.390718,0.9010,-0.114139,0.956241
219,2021-12,7.402683,0.8330,-0.362188,0.954434
220,2022-01,7.409585,0.5190,0.711944,0.978767
221,2022-02,7.401762,0.5180,-0.115408,0.979315


In [244]:
a = preprocess()
a.daily_preprocessing('Canada')

,date,ex,int_spread,inf_spread,m1_spread
4571,2022-02-17,6.867693,0.240,-0.449674,1.488475
4572,2022-02-18,6.867881,0.240,-0.449674,1.488475
4573,2022-02-21,6.861596,0.247,-0.449674,1.488475
4574,2022-02-22,6.861837,0.231,-0.449674,1.488475
4575,2022-02-23,6.861460,0.213,-0.449674,1.488475
4576,2022-02-24,6.864775,0.230,-0.449674,1.488475
4577,2022-02-25,6.865317,0.231,-0.449674,1.488475
4578,2022-02-28,6.867090,0.256,-0.449674,1.488475
4579,2022-03-02,6.875945,0.217,-0.708770,1.471268
4580,2022-03-03,6.883575,0.220,-0.708770,1.471268
